In [15]:
import mlflow
import mlflow.sklearn
import dagshub
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.feature_extraction import DictVectorizer
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
from mlflow import MlflowClient
from datetime import datetime
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import mlflow
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
from mlflow import MlflowClient


# Inicializar DagsHub y MLflow
dagshub.init(url="https://dagshub.com/JuanPab2009/nys-taxi-time-prediction", mlflow=True)

MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="nyc-taxi-experiment-challenger")

Initialized MLflow to track repo "JuanPab2009/nys-taxi-time-prediction"

Repository JuanPab2009/nys-taxi-time-prediction initialized!

<Experiment: artifact_location='mlflow-artifacts:/508414111c7848b9aa19a2969e8ed6c8', creation_time=1728434351137, experiment_id='1', last_update_time=1728434351137, lifecycle_stage='active', name='nyc-taxi-experiment-challenger', tags={}>

In [2]:
# Crear el directorio si no existe
!mkdir -p ../data

# Descargar el archivo de marzo
!curl -o ../data/green_tripdata_2024-03.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-03.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1340k  100 1340k    0     0  2281k      0 --:--:-- --:--:-- --:--:-- 2283k


In [3]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df

# Cargar el nuevo dataset
df_marzo = read_dataframe('../data/green_tripdata_2024-03.parquet')

# Guardar en MLflow storage
training_dataset_marzo = mlflow.data.from_pandas(df_marzo, name="green_tripdata_2024-03")
mlflow.log_artifact("../data/green_tripdata_2024-03.parquet", artifact_path="data")

In [4]:
# Feature Engineering
def feature_engineering(df):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    return df

df_train = read_dataframe('../data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2024-02.parquet')

df_train = feature_engineering(df_train)
df_val = feature_engineering(df_val)

# One Hot Encoding
categorical = ['PU_DO']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

y_train = df_train['duration'].values
y_val = df_val['duration'].values

In [5]:
def objective(params, model_type='gb'):
    with mlflow.start_run(nested=True):
        mlflow.set_tag("model_family", model_type)
        mlflow.log_params(params)
        
        if model_type == 'gb':
            model = GradientBoostingRegressor(**params)
        elif model_type == 'rf':
            model = RandomForestRegressor(**params)
        else:
            raise ValueError("Unsupported model type")
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)
        
        mlflow.log_metric("rmse", rmse)
        mlflow.sklearn.log_model(model, artifact_path="model")
        
    return {'loss': rmse, 'status': STATUS_OK}

In [6]:
search_space_gb = {
    'n_estimators': scope.int(hp.quniform('n_estimators', 100, 1000, 100)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'max_depth': scope.int(hp.quniform('max_depth', 3, 10, 1)),
    'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 10, 1)),
    'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf', 1, 4, 1)),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'random_state': 42
}

In [17]:
search_space_rf = {
    'n_estimators': scope.int(hp.quniform('n_estimators', 100, 1000, 100)),
    'max_depth': scope.int(hp.quniform('max_depth', 5, 30, 5)),
    'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 20, 2)),
    'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf', 1, 10, 1)),
    'max_features': hp.choice('max_features', ['sqrt', 'log2']),
    'random_state': 42
}

In [8]:
with mlflow.start_run(run_name="GradientBoosting_Hyperopt", nested=True):
    trials_gb = Trials()
    best_gb = fmin(
        fn=lambda params: objective(params, model_type='gb'),
        space=search_space_gb,
        algo=tpe.suggest,
        max_evals=20,
        trials=trials_gb
    )
    
    # Convertir los parámetros óptimos
    best_gb['n_estimators'] = int(best_gb['n_estimators'])
    best_gb['max_depth'] = int(best_gb['max_depth'])
    best_gb['min_samples_split'] = int(best_gb['min_samples_split'])
    best_gb['min_samples_leaf'] = int(best_gb['min_samples_leaf'])
    best_gb['random_state'] = 42
    
    mlflow.log_params(best_gb)
    
    # Entrenar el modelo final
    gb_model = GradientBoostingRegressor(**best_gb)
    gb_model.fit(X_train, y_train)
    y_pred_gb = gb_model.predict(X_val)
    rmse_gb = root_mean_squared_error(y_val, y_pred_gb)
    mlflow.log_metric("rmse_final", rmse_gb)
    mlflow.sklearn.log_model(gb_model, artifact_path="gb_model")

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

2024/10/08 18:40:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

2024/10/08 18:40:28 INFO mlflow.tracking._tracking_service.client: 🏃 View run salty-doe-263 at: https://dagshub.com/JuanPab2009/nys-taxi-time-prediction.mlflow/#/experiments/1/runs/450500fc3d974a8ead47167cf939736a.

2024/10/08 18:40:28 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/JuanPab2009/nys-taxi-time-prediction.mlflow/#/experiments/1.



  5%|▌         | 1/20 [00:18<05:58, 18.89s/trial, best loss: 5.193053785948702]

2024/10/08 18:40:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

2024/10/08 18:40:38 INFO mlflow.tracking._tracking_service.client: 🏃 View run charming-grub-906 at: https://dagshub.com/JuanPab2009/nys-taxi-time-prediction.mlflow/#/experiments/1/runs/adce69ad99e2451dbfd8f2cd50c26f4b.

2024/10/08 18:40:38 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/JuanPab2009/nys-taxi-time-prediction.mlflow/#/experiments/1.



 10%|█         | 2/20 [00:28<04:00, 13.37s/trial, best loss: 5.193053785948702]

...

2024/10/08 18:44:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

2024/10/08 18:44:57 INFO mlflow.tracking._tracking_service.client: 🏃 View run melodic-zebra-731 at: https://dagshub.com/JuanPab2009/nys-taxi-time-prediction.mlflow/#/experiments/1/runs/13226cdf306343e2bf4a295be166ba05.

2024/10/08 18:44:57 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/JuanPab2009/nys-taxi-time-prediction.mlflow/#/experiments/1.



 95%|█████████▌| 19/20 [04:47<00:13, 13.21s/trial, best loss: 5.138932505351716]

2024/10/08 18:45:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

2024/10/08 18:45:22 INFO mlflow.tracking._tracking_service.client: 🏃 View run thundering-gnat-206 at: https://dagshub.com/JuanPab2009/nys-taxi-time-prediction.mlflow/#/experiments/1/runs/9f1921131c76410db02f3d78b53fdb87.

2024/10/08 18:45:22 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/JuanPab2009/nys-taxi-time-prediction.mlflow/#/experiments/1.



100%|██████████| 20/20 [05:11<00:00, 15.60s/trial, best loss: 5.138932505351716]


2024/10/08 18:45:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/08 18:45:45 INFO mlflow.tracking._tracking_service.client: 🏃 View run GradientBoosting_Hyperopt at: https://dagshub.com/JuanPab2009/nys-taxi-time-prediction.mlflow/#/experiments/1/runs/ed99dc745da54b18b609cac224202bc1.
2024/10/08 18:45:45 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/JuanPab2009/nys-taxi-time-prediction.mlflow/#/experiments/1.


In [13]:
with mlflow.start_run(run_name="RandomForest_Hyperopt", nested=True):
    trials_rf = Trials()
    best_rf = fmin(
        fn=lambda params: objective(params, model_type='rf'),
        space=search_space_rf,
        algo=tpe.suggest,
        max_evals=20,
        trials=trials_rf
    )
    
    # Convertir los parámetros óptimos
    best_rf['n_estimators'] = int(best_rf['n_estimators'])
    best_rf['max_depth'] = int(best_rf['max_depth'])
    best_rf['min_samples_split'] = int(best_rf['min_samples_split'])
    best_rf['min_samples_leaf'] = int(best_rf['min_samples_leaf'])
    best_rf['max_features'] = ['auto', 'sqrt', 'log2'][best_rf['max_features']]
    best_rf['random_state'] = 42
    
    mlflow.log_params(best_rf)
    
    # Entrenar el modelo final
    rf_model = RandomForestRegressor(**best_rf)
    rf_model.fit(X_train, y_train)
    y_pred_rf = rf_model.predict(X_val)
    rmse_rf = root_mean_squared_error(y_val, y_pred_rf)
    mlflow.log_metric("rmse_final", rmse_rf)
    mlflow.sklearn.log_model(rf_model, artifact_path="rf_model")

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

2024/10/08 19:34:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

2024/10/08 19:34:14 INFO mlflow.tracking._tracking_service.client: 🏃 View run blushing-gnu-638 at: https://dagshub.com/JuanPab2009/nys-taxi-time-prediction.mlflow/#/experiments/1/runs/03bd95386abb41f6a12ce705fe6280cb.

2024/10/08 19:34:14 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/JuanPab2009/nys-taxi-time-prediction.mlflow/#/experiments/1.



  5%|▌         | 1/20 [00:08<02:41,  8.50s/trial, best loss: 7.998198875703363]

2024/10/08 19:34:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

2024/10/08 19:34:22 INFO mlflow.tracking._tracking_service.client: 🏃 View run illustrious-eel-339 at: https://dagshub.com/JuanPab2009/nys-taxi-time-prediction.mlflow/#/experiments/1/runs/845f5ffbd6dd4fb7b6cd697d82373b67.

2024/10/08 19:34:22 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/JuanPab2009/nys-taxi-time-prediction.mlflow/#/experiments/1.



 10%|█         | 2/20 [00:16<02:27,  8.20s/trial, best loss: 7.998198875703363]

...

 95%|█████████▌| 19/20 [02:48<00:10, 10.96s/trial, best loss: 7.537639066755954]

2024/10/08 19:36:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

2024/10/08 19:37:01 INFO mlflow.tracking._tracking_service.client: 🏃 View run valuable-gnat-27 at: https://dagshub.com/JuanPab2009/nys-taxi-time-prediction.mlflow/#/experiments/1/runs/ef312375951c47569d5727913ca4e31e.

2024/10/08 19:37:01 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/JuanPab2009/nys-taxi-time-prediction.mlflow/#/experiments/1.



100%|██████████| 20/20 [02:54<00:00,  8.75s/trial, best loss: 7.537639066755954]


2024/10/08 19:37:01 INFO mlflow.tracking._tracking_service.client: 🏃 View run RandomForest_Hyperopt at: https://dagshub.com/JuanPab2009/nys-taxi-time-prediction.mlflow/#/experiments/1/runs/989a9d704c5a4d11ba9fe3fed3bc5bc8.
2024/10/08 19:37:01 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/JuanPab2009/nys-taxi-time-prediction.mlflow/#/experiments/1.


InvalidParameterError: The 'max_features' parameter of RandomForestRegressor must be an int in the range [1, inf), a float in the range (0.0, 1.0], a str among {'sqrt', 'log2'} or None. Got 'auto' instead.

In [19]:
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

# Inicializar el cliente de MLflow
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

# Nombre del experimento
experiment_name = "nyc-taxi-experiment-challenger"

# Obtener el experimento
experiment = client.get_experiment_by_name(experiment_name)

if not experiment:
    print(f"El experimento '{experiment_name}' no existe.")
else:
    print(f"Experiment ID: {experiment.experiment_id}")
    
    # Buscar runs principales (sin parent_id)
    runs = client.search_runs(
        experiment_ids=experiment.experiment_id,
        filter_string="",
        run_view_type=ViewType.ACTIVE_ONLY,
        max_results=1000
    )
    
    # Filtrar runs que no tienen parent_id y tienen 'rmse_final'
    primary_runs = [
        run for run in runs 
        if run.info.parent_run_id is None and 'rmse_final' in run.data.metrics
    ]
    
    if not primary_runs:
        print("No hay runs principales con la métrica 'rmse_final'.")
    else:
        # Encontrar el run con el menor RMSE
        best_run = min(primary_runs, key=lambda run: run.data.metrics["rmse_final"])
        best_run_id = best_run.info.run_id
        best_model_uri = f"runs:/{best_run_id}/model"
        
        print(f"El mejor run ID es: {best_run_id} con rmse_final: {best_run.data.metrics['rmse_final']}")
        print(f"Modelo registrado en: {best_model_uri}")

Experiment ID: 1


AttributeError: 'RunInfo' object has no attribute 'parent_run_id'

In [ ]:
result = mlflow.register_model(
    model_uri=best_model_uri,
    name="nyc-taxi-model"
)

In [ ]:
from mlflow import MlflowClient
from datetime import datetime

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

# Obtener la versión del modelo registrado
model_version = result.version

# Asignar el alias "challenger"
client.set_registered_model_alias(
    name="nyc-taxi-model",
    alias="challenger",
    version=model_version
)

# Actualizar la descripción de la versión
date = datetime.today()
client.update_model_version(
    name="nyc-taxi-model",
    version=model_version,
    description=f"The model version {model_version} was set as challenger on {date}"
)

In [20]:
import mlflow
from mlflow import MlflowClient
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import pickle
import pathlib

# Definir la función objetivo para Random Forest
def objective_rf(params):
    with mlflow.start_run(nested=True):
        mlflow.set_tag("model_family", "random_forest")
        mlflow.log_params(params)
        
        # Crear el modelo con los parámetros actuales
        model = RandomForestRegressor(
            n_estimators=params['n_estimators'],
            max_depth=params['max_depth'],
            min_samples_split=params['min_samples_split'],
            min_samples_leaf=params['min_samples_leaf'],
            max_features=params['max_features'],
            random_state=params['random_state'],
            n_jobs=-1
        )
        
        # Entrenar el modelo
        model.fit(X_train, y_train)
        
        # Predecir en el conjunto de validación
        y_pred = model.predict(X_val)
        
        # Calcular el RMSE
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        
        # Loguear la métrica intermedia
        mlflow.log_metric("rmse", rmse)
        
        # Loguear el modelo
        mlflow.sklearn.log_model(model, artifact_path="rf_model")
        
    return {'loss': rmse, 'status': STATUS_OK}

# Definir el espacio de búsqueda sin 'auto'
search_space_rf = {
    'n_estimators': scope.int(hp.quniform('n_estimators', 100, 1000, 100)),
    'max_depth': scope.int(hp.quniform('max_depth', 5, 30, 5)),
    'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 20, 2)),
    'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf', 1, 10, 1)),
    'max_features': hp.choice('max_features', ['sqrt', 'log2']),
    'random_state': 42
}

# Ejecutar la optimización
with mlflow.start_run(run_name="RandomForest_Hyperopt", nested=True):
    trials_rf = Trials()
    best_rf = fmin(
        fn=objective_rf,
        space=search_space_rf,
        algo=tpe.suggest,
        max_evals=20,
        trials=trials_rf
    )
    
    # Convertir los parámetros óptimos
    best_rf['n_estimators'] = int(best_rf['n_estimators'])
    best_rf['max_depth'] = int(best_rf['max_depth'])
    best_rf['min_samples_split'] = int(best_rf['min_samples_split'])
    best_rf['min_samples_leaf'] = int(best_rf['min_samples_leaf'])
    best_rf['random_state'] = 42  # Ya está definido
    
    # Obtener el valor seleccionado para 'max_features'
    max_features_options = ['sqrt', 'log2']
    best_rf['max_features'] = max_features_options[best_rf['max_features']]
    
    # Loguear los mejores parámetros
    mlflow.log_params(best_rf)
    
    # Entrenar el modelo final con los mejores parámetros
    final_rf_model = RandomForestRegressor(
        n_estimators=best_rf['n_estimators'],
        max_depth=best_rf['max_depth'],
        min_samples_split=best_rf['min_samples_split'],
        min_samples_leaf=best_rf['min_samples_leaf'],
        max_features=best_rf['max_features'],
        random_state=best_rf['random_state'],
        n_jobs=-1
    )
    
    final_rf_model.fit(X_train, y_train)
    y_pred_rf = final_rf_model.predict(X_val)
    rmse_rf = mean_squared_error(y_val, y_pred_rf, squared=False)
    
    # Loguear la métrica final
    mlflow.log_metric("rmse_final", rmse_rf)
    
    # Loguear el modelo final
    mlflow.sklearn.log_model(final_rf_model, artifact_path="rf_final_model")
    
    print(f"Mejores parámetros para Random Forest: {best_rf}")
    print(f"RMSE final para Random Forest: {rmse_rf}")

# Seleccionar el mejor run basado en 'rmse_final'
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

experiment_name = "nyc-taxi-experiment-challenger"

experiment = client.get_experiment_by_name(experiment_name)

if not experiment:
    print(f"El experimento '{experiment_name}' no existe.")
else:
    print(f"Experiment ID: {experiment.experiment_id}")
    
    runs = client.search_runs(
        experiment_ids=experiment.experiment_id,
        filter_string="",
        run_view_type=ViewType.ACTIVE_ONLY,
        max_results=1000
    )
    
    runs_with_rmse_final = [run for run in runs if 'rmse_final' in run.data.metrics]
    
    if not runs_with_rmse_final:
        print("No hay runs con la métrica 'rmse_final'.")
    else:
        try:
            best_run = min(runs_with_rmse_final, key=lambda run: run.data.metrics["rmse_final"])
            best_run_id = best_run.info.run_id
            best_model_uri = f"runs:/{best_run_id}/rf_final_model"
            
            print(f"El mejor run ID es: {best_run_id} con rmse_final: {best_run.data.metrics['rmse_final']}")
            print(f"Modelo registrado en: {best_model_uri}")
        except ValueError:
            print("No se pudo determinar el mejor run debido a valores inválidos.")

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

/Users/juanpabloblanco/Documents/nys-taxi-time-prediction/.venv/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

2024/10/08 19:45:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

2024/10/08 19:45:46 INFO mlflow.tracking._tracking_service.client: 🏃 View run chill-fish-217 at: https://dagshub.com/JuanPab2009/nys-taxi-time-prediction.mlflow/#/experiments/1/runs/8a8440125a6e434b837676cf217f373f.

2024/10/08 19:45:46 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/JuanPab2009/nys-taxi-time-prediction.mlflow/#/experiments/1.



  5%|▌         | 1/20 [00:06<02:03,  6.47s/trial, best loss: 8.43474804615386]

/Users/juanpabloblanco/Documents/nys-taxi-time-prediction/.venv/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

2024/10/08 19:45:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

2024/10/08 19:45:51 INFO mlflow.tracking._tracking_service.client: 🏃 View run abrasive-crab-904 at: https://dagshub.com/JuanPab2009/nys-taxi-time-prediction.mlflow/#/experiments/1/runs/0f8750a1c59e435896eec8bd8e887b09.

2024/10/08 19:45:51 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/JuanPab2009/nys-taxi-time-prediction.mlflow/#/experiments/1.



 10%|█         | 2/20 [00:11<01:43,  5.76s/trial, best loss: 8.43474804615386]

...

 95%|█████████▌| 19/20 [02:03<00:07,  7.33s/trial, best loss: 7.546235282965409]

/Users/juanpabloblanco/Documents/nys-taxi-time-prediction/.venv/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

2024/10/08 19:47:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.

2024/10/08 19:47:49 INFO mlflow.tracking._tracking_service.client: 🏃 View run salty-shark-498 at: https://dagshub.com/JuanPab2009/nys-taxi-time-prediction.mlflow/#/experiments/1/runs/2f065044cd7a43539851604091fe3d76.

2024/10/08 19:47:49 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/JuanPab2009/nys-taxi-time-prediction.mlflow/#/experiments/1.



100%|██████████| 20/20 [02:09<00:00,  6.47s/trial, best loss: 7.546235282965409]


/Users/juanpabloblanco/Documents/nys-taxi-time-prediction/.venv/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2024/10/08 19:47:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/08 19:47:58 INFO mlflow.tracking._tracking_service.client: 🏃 View run RandomForest_Hyperopt at: https://dagshub.com/JuanPab2009/nys-taxi-time-prediction.mlflow/#/experiments/1/runs/e0b149d7af644caf8dee7736ccb252a8.
2024/10/08 19:47:58 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/JuanPab2009/nys-taxi-time-prediction.mlflow/#/experiments/1.


Mejores parámetros para Random Forest: {'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 800, 'random_state': 42}
RMSE final para Random Forest: 7.546235282965409
Experiment ID: 1
El mejor run ID es: ed99dc745da54b18b609cac224202bc1 con rmse_final: 5.138932505351716
Modelo registrado en: runs:/ed99dc745da54b18b609cac224202bc1/rf_final_model


In [ ]:
import mlflow.pyfunc

# Cargar el modelo champion
model_champion = mlflow.pyfunc.load_model("models:/nyc-taxi-model@champion")

# Cargar el modelo challenger
model_challenger = mlflow.pyfunc.load_model("models:/nyc-taxi-model@challenger")